### Desarrollo de algoritmos de ensamblado

## Implementación de paralelismo con Dask

## **Conjunto de datos sobre diabetes**
Este conjunto de datos proviene del Instituto Nacional de Diabetes y
Enfermedades Digestivas y Renales. El objetivo del conjunto de datos es predecir de forma diagnóstica si un paciente tiene diabetes,
basándose en ciertas mediciones de diagnóstico incluidas en el conjunto de datos.
En particular, todos los pacientes aquí son mujeres de al menos 21 años de edad de ascendencia india 

Información sobre los atributos del conjunto de datos -

Embarazos: Para expresar el Número de embarazos

Glucosa: Para expresar el nivel de glucosa en sangre.

BloodPressure: Para expresar la medición de la presión arterial.

SkinThickness: Para expresar el grosor de la piel.

Insulina: Para expresar el nivel de insulina en sangre.

IMC: Para expresar el índice de masa corporal.

DiabetesPedigreeFunción: Expresar el porcentaje de Diabetes.

Edad: Para expresar la edad.

Resultado: Para expresar el resultado final 1 es Sí y 0 es No

https://www.kaggle.com/datasets/akshaydattatraykhare/diabetes-dataset

#### Predice si tiene diabetes sabiendo su nivel de glucosa,  índice de masa corporal y edad

In [29]:
# Iniciar un cliente Dask
from dask.distributed import Client
client = Client(n_workers=4)
client



c:\Users\Asus\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54286 instead
  warnings.warn(
c:\Users\Asus\AppData\Local\Programs\Python\Python39\lib\contextlib.py:124: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.16s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:54286/status,
Dashboard: http://127.0.0.1:54286/status,Workers: 4
Total threads: 8,Total memory: 5.95 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54292,Workers: 4
Dashboard: http://127.0.0.1:54286/status,Total threads: 8
Started: Just now,Total memory: 5.95 GiB
Comm: tcp://127.0.0.1:54326,Total threads: 2
Dashboard: http://127.0.0.1:54327/status,Memory: 1.49 GiB
Nanny: tcp://127.0.0.1:54295,


In [30]:
#Revisamos el contenido de la data
import dask.dataframe as dd

data1 = dd.read_csv('diabetes.csv')
data1.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [31]:
from dask_ml.model_selection import train_test_split
import time
X = data1[['Glucose','BMI','Age']]  # Seleccionamos las características adecuadas
y = data1['Outcome']


# Medir el tiempo de ejecución
start_time0 = time.time()

# Divide tus datos en conjuntos de entrenamiento y prueba, utilizamos stratify para distribuir bien la cantidades de datos para las clases
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)#,stratify=y


# Calcular el tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time0

print("Tiempo de ejecución: {:.2f} segundos".format(execution_time))#,stratify=y

Tiempo de ejecución: 0.08 segundos


c:\Users\Asus\AppData\Local\Programs\Python\Python39\lib\site-packages\dask_ml\model_selection\_split.py:464: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [32]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((<dask_expr.expr.Scalar: expr=(SplitTake(frame=Split(frame=ReadCSV(38c3c0c)[['Glucose', 'BMI', 'Age']], frac=[0.8, 0.2], random_state=1608637542, shuffle=False), i=0, ndim=2)).size() // 3, dtype=int32>,
  3),
 (<dask_expr.expr.Scalar: expr=(SplitTake(frame=Split(frame=ReadCSV(38c3c0c)['Outcome'], frac=[0.8, 0.2], random_state=1608637542, shuffle=False), i=0, ndim=1)).size(), dtype=int32>,),
 (<dask_expr.expr.Scalar: expr=(SplitTake(frame=Split(frame=ReadCSV(38c3c0c)[['Glucose', 'BMI', 'Age']], frac=[0.8, 0.2], random_state=1608637542, shuffle=False), i=1, ndim=2)).size() // 3, dtype=int32>,
  3),
 (<dask_expr.expr.Scalar: expr=(SplitTake(frame=Split(frame=ReadCSV(38c3c0c)['Outcome'], frac=[0.8, 0.2], random_state=1608637542, shuffle=False), i=1, ndim=1)).size(), dtype=int32>,))

In [40]:
pip install --upgrade dask-xgboost

In [44]:
from dask_ml.xgboost import XGBClassifier

# Crear y entrenar el modelo
gb = XGBClassifier(objective='binary:logistic', random_state=42)
gb.fit(X_train, y_train)

X_test_dask_array = X_test.to_dask_array()

# Ensure type conversion is successful (optional)
print(type(X_test_dask_array))  # Check the type after conversion

# Inspect Dask array contents (optional)
print(X_test_dask_array.head())  # View a few samples

# Try alternative prediction methods if necessary
# Refer to dask_xgboost documentation for specific methods

predictions = gb.predict(X_test_dask_array).compute()
# Realizar predicciones
#gb.predict(X_test).compute()

<class 'dask.array.core.Array'>


AttributeError: 'Array' object has no attribute 'head'

In [47]:
X_test_dask_array = X_test_dask_array.compute_chunk_sizes()
sample = X_test_dask_array[:2].compute()  # Now it might work
print(sample)


In [36]:
print(type(gb))

<class 'dask_xgboost.core.XGBClassifier'>


In [10]:
import dask.dataframe as dd
import dask.array as da
from dask_ml.model_selection import train_test_split
import dask_xgboost as dxgb
import xgboost as xgb

# Leer los datos en un DataFrame de Dask
df_dask = dd.read_csv('diabetes.csv')

# Separar características y etiquetas
X = df_dask[['Glucose','BMI','Age']]  # Seleccionamos las características adecuadas
y = df_dask['Outcome']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Convertir los datos a Dask Arrays
X_train = X_train.to_dask_array(lengths=True)
X_test = X_test.to_dask_array(lengths=True)
y_train = y_train.to_dask_array(lengths=True)
y_test = y_test.to_dask_array(lengths=True)

# Configurar parámetros de XGBoost
params = {
    'objective': 'binary:logistic',
    'random_state': 42
}

# Entrenar el modelo con dask-xgboost
dtrain = dxgb.DaskDMatrix(client, X_train, y_train)
model = dxgb.train(client, params, dtrain, num_boost_round=100)

# Predecir con el modelo entrenado
dtest = dxgb.DaskDMatrix(client, X_test)
predictions = dxgb.predict(client, model, dtest)
predictions = predictions.compute()


AttributeError: module 'dask_xgboost' has no attribute 'DaskDMatrix'

In [7]:
predictions_xg = model.predict(X_test).compute()

TypeError: isinstance() arg 2 must be a type or tuple of types

### **Random Forest:**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Crear el clasificador de Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf_classifier.fit(X_train, y_train)

# Predicción en el conjunto de prueba
predictions_rf = rf_classifier.predict(X_test)

In [ ]:
y_test.head(10)

In [ ]:
predictions_rf[:10]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions_rf)
accuracy

In [ ]:
from sklearn.metrics import precision_score
#Calcula la precisión para cada clase individualmente y luego toma la media.
precision = precision_score(y_test, predictions_rf, average='macro')
print(precision)


### **Gradient Boosting:**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Crear el clasificador de Gradient Boosting
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# Entrenar el modelo
gb_classifier.fit(X_train, y_train)

# Predicción en el conjunto de prueba
predictions_gb = gb_classifier.predict(X_test)


In [ ]:
y_test.head(10)

In [ ]:
predictions_gb[:10]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions_gb)
accuracy

In [ ]:
from sklearn.metrics import precision_score
#Calcula la precisión para cada clase individualmente y luego toma la media.
precision = precision_score(y_test, predictions_gb, average='macro')
print(precision)

### **XGBoost:**

https://ml.dask.org/joblib.html

In [69]:
import xgboost as xgb

# Crear el clasificador de XGBoost
xgb_classifier = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

# Medir el tiempo de ejecución
start_time0 = time.time()
# Entrenar el modelo
xgb_classifier.fit(X_train, y_train)

# Calcular el tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time0

print("Tiempo de ejecución: {:.2f} segundos".format(execution_time))
# Predicción en el conjunto de prueba
predictions_xgb = xgb_classifier.predict(X_test)

Tiempo de ejecución: 7.98 segundos


In [ ]:
predictions_xgb

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions_xgb)
accuracy

In [ ]:
client.close()

In [ ]:
#import xgboost as xgb
from dask_ml.xgboost import XGBClassifier
# Crear el clasificador de XGBoost
xgb_classifier = XGBClassifier(objective="binary:logistic", random_state=42)

# Entrenar el modelo
xgb_classifier.fit(X_train, y_train)

# Predicción en el conjunto de prueba
predictions_xgb = xgb_classifier.predict(X_test)


In [ ]:
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
from dask_ml.xgboost import XGBClassifier
from dask.distributed import Client, wait
import webbrowser
import numpy as np
import dask.array as da

# Iniciar un cliente Dask
client = Client()

# Abrir automáticamente el tablero en el navegador
webbrowser.open_new_tab(client.dashboard_link)

# Leer el archivo CSV con Dask
data1 = dd.read_csv('diabetes.csv')

# Dividir los datos en características (X) y etiquetas (y)
X = data1[['Glucose', 'BMI', 'Age']]
y = data1['Outcome']

# Dividir los datos en entrenamiento y prueba usando Dask-ML
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Crear el clasificador de XGBoost
xgb_classifier = XGBClassifier(objective="binary:logistic", random_state=42)

# Entrenar el modelo
xgb_classifier.fit(X_train, y_train)

# Predicción en el conjunto de prueba
predictions_xgb = xgb_classifier.predict(X_test)

predictions_xgb 
# Cerrar el cliente Dask al finalizar
client.close()


In [ ]:
y_test.head(10)

In [ ]:
predictions_xgb[:10]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions_xgb)
accuracy

In [ ]:
from sklearn.metrics import precision_score
#Calcula la precisión para cada clase individualmente y luego toma la media.
precision = precision_score(y_test, predictions_xgb, average='macro')
print(precision)


### **Combinar múltiples modelos**

https://inteligencia-artificial.dev/ensemble-learning-python/

In [ ]:
from sklearn.ensemble import VotingClassifier

# Crear un clasificador de votación suave
voting_classifier = VotingClassifier(estimators=[
    ('rf', rf_classifier), ('xgb', xgb_classifier)], voting='soft')

# Entrenar el modelo de votación
voting_classifier.fit(X_train, y_train)

# Predicción en el conjunto de prueba
predictions_voting = voting_classifier.predict(X_test)


In [ ]:
y_test.head(14)

In [ ]:
predictions_voting[:14]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions_voting)
accuracy

In [ ]:
from sklearn.metrics import precision_score
#Calcula la precisión para cada clase individualmente y luego toma la media.
precision = precision_score(y_test, predictions_voting, average='macro')
print(precision)